In [3]:
import numpy as np
import random
from operator import itemgetter
from random import randint

<h2>Funciones de prueba</h2>

In [4]:
def calculoCM(n):
    k = (n*(n**2 + 1))/2
    return k 

In [5]:
def conteoExitos(cuadrado = np.array([])):

    n = int(len(cuadrado)**0.5)

    k = calculoCM(n)

    matriz = cuadrado.reshape((n,n))

    sumatoria_columnas = np.sum(matriz, axis=0)
    sumatoria_filas = np.sum(matriz, axis=1)
    diagonal_principal = np.trace(matriz)
    diagonal_secundaria = np.trace(matriz[::-1])

    valores = np.concatenate((sumatoria_columnas, sumatoria_filas, np.array([diagonal_principal, diagonal_secundaria])), axis = 0)

    numExitos = 0

    for i in valores:
        if k == i:
            numExitos+=1

    return numExitos

In [6]:
def errorMinimo(cuadrado = np.array([])):

    n = int(len(cuadrado)**0.5)

    k = calculoCM(n)

    matriz = cuadrado.reshape((n,n))

    sumatoria_columnas = np.sum(matriz, axis=0)
    sumatoria_filas = np.sum(matriz, axis=1)
    diagonal_principal = np.trace(matriz)
    diagonal_secundaria = np.trace(matriz[::-1])

    valores = np.concatenate((sumatoria_columnas, sumatoria_filas, np.array([diagonal_principal, diagonal_secundaria])), axis = 0)

    error = 0

    for i in valores:
        error += np.abs((k-i))

    return error

In [7]:
# Generar un np.array de valores del 1 al 9 en orden
arr = np.arange(1, 10)

# Permutar aleatoriamente los valores
np.random.shuffle(arr)

print(arr.reshape((3,3)))

print(conteoExitos(cuadrado = arr))
print(errorMinimo(cuadrado = arr))

[[5 2 4]
 [3 8 9]
 [7 1 6]]
1
26.0


<h2>Estrategia de selección de padres</h2>

<h3>La ruleta</h3>

In [8]:
def seleccion_ruleta(poblacion, fitness):
    # Normalizar el fitness
    probabilidad = fitness / np.sum(fitness)
    probAcumulada = np.cumsum(probabilidad)
    r = np.random.uniform(0, 1, size=len(poblacion))
    # Seleccionar los padres
    padres = list()
    for i in range(len(poblacion)):
        index = np.where(probAcumulada >= r[i])[0][0]
        padres.append(index)
        
    return padres

<h2>Operador de cruza (PMX)</h2>

In [9]:
def pmx(parent1, parent2):
    size = len(parent1)
    child1 = np.zeros(size, dtype=np.uint32)
    child2 = np.zeros(size, dtype=np.uint32)

    # Seleccionar dos puntos aleatorios
    point1 = np.random.randint(0, size)
    point2 = np.random.randint(0, size - 1)
    if point2 >= point1:
        point2 += 1

    # Copiar los segmentos seleccionados de los padres a los hijos
    child1[point1:point2] = parent1[point1:point2]
    child2[point1:point2] = parent2[point1:point2]

    # Completar los hijos con los valores faltantes
    for i in range(size):
        if not parent2[i] in child1:
            for j in range(size):
                if child1[j] == 0:
                    child1[j] = parent2[i]
                    break
        if not parent1[i] in child2:
            for j in range(size):
                if child2[j] == 0:
                    child2[j] = parent1[i]
                    break

    return child1, child2


<h2>Operador de mutación</h2>

In [10]:
def mutacion(poblacion, porcMuta, fPrueba):  

  cromosomas = [sublista[0] for sublista in poblacion]

  for i in range(len(cromosomas)):
    if porcMuta >= np.random.uniform(0,1): 
      
      inicio = random.randint(0, len(cromosomas[i])-2)
      final = random.randint(inicio+1, len(cromosomas[i])-1)

      aux = cromosomas[i][inicio]
      cromosomas[i][inicio] = cromosomas[i][final]
      cromosomas[i][final] = aux


  out = list()

  for i in cromosomas:
      if fPrueba == 0:
          fx = conteoExitos(i)
      elif fPrueba == 1:
          fx = errorMinimo(i)

      out.append([i, fx])

  return out

<h2>Estructura del algoritmo genetico</h2>

<h3>Inicialización la población</h3>

In [11]:
def poblacionInicial(tam, n, mode):

  if mode == 0:
    function = conteoExitos
  elif mode == 1:
    function = errorMinimo

  poblacion = list()
  for i in range(0, tam-1): 
    arr = np.arange(1, (n**2)+1)
    np.random.shuffle(arr)
   
    fx = round(function(arr),4)
    poblacion.append([arr, fx])
    
  return poblacion

In [12]:
poblacionInicial(10,3,1)

[[array([3, 7, 2, 9, 4, 1, 8, 6, 5]), 26.0],
 [array([7, 3, 5, 8, 4, 2, 9, 6, 1]), 26.0],
 [array([5, 8, 4, 2, 9, 6, 3, 7, 1]), 27.0],
 [array([8, 1, 9, 4, 5, 7, 2, 6, 3]), 18.0],
 [array([7, 4, 9, 3, 6, 8, 2, 5, 1]), 23.0],
 [array([1, 8, 7, 3, 2, 4, 6, 5, 9]), 25.0],
 [array([7, 8, 5, 1, 4, 2, 6, 3, 9]), 23.0],
 [array([3, 6, 5, 9, 7, 2, 1, 4, 8]), 15.0],
 [array([4, 5, 6, 8, 9, 7, 2, 3, 1]), 25.0]]

In [13]:
poblacionInicial(10,3,1)

[[array([1, 6, 2, 9, 5, 4, 3, 7, 8]), 24.0],
 [array([1, 3, 9, 6, 4, 2, 7, 8, 5]), 22.0],
 [array([8, 2, 5, 3, 1, 6, 9, 4, 7]), 27.0],
 [array([4, 9, 5, 1, 6, 8, 3, 2, 7]), 23.0],
 [array([9, 4, 5, 3, 7, 1, 8, 2, 6]), 30.0],
 [array([9, 1, 3, 4, 5, 2, 8, 6, 7]), 31.0],
 [array([5, 1, 8, 9, 7, 4, 2, 6, 3]), 14.0],
 [array([3, 1, 6, 2, 7, 9, 8, 5, 4]), 25.0],
 [array([4, 5, 6, 7, 2, 1, 3, 8, 9]), 16.0]]

In [14]:
def seleccionPadres(poblacion):
    padres = [sublista[0] for sublista in poblacion]
    fitness = [sublista[1] for sublista in poblacion]

    return seleccion_ruleta(padres, fitness)

In [15]:
a = poblacionInicial(10,3,1)
b = seleccionPadres(a)
print(a)
print(b)

[[array([1, 4, 7, 8, 2, 9, 3, 6, 5]), 30.0], [array([6, 5, 4, 3, 1, 8, 9, 2, 7]), 22.0], [array([3, 4, 7, 9, 1, 6, 5, 2, 8]), 23.0], [array([8, 3, 6, 2, 9, 4, 1, 7, 5]), 20.0], [array([3, 2, 4, 9, 7, 6, 5, 1, 8]), 28.0], [array([1, 4, 5, 8, 7, 9, 3, 6, 2]), 29.0], [array([2, 5, 8, 7, 4, 9, 3, 6, 1]), 24.0], [array([1, 4, 6, 9, 2, 3, 8, 7, 5]), 24.0], [array([9, 8, 2, 5, 7, 4, 3, 6, 1]), 31.0]]
[5, 6, 6, 5, 5, 3, 6, 1, 5]


<h3>Cruza de padres con cierta probabilidad</h3>

In [16]:
def cruza(padres, indices, prob_cruza, fPrueba):

    cromosomas = [sublista[0] for sublista in padres]

    hijos = list()
    i = 0

    while i < len(cromosomas):

        try:
            padre1 = cromosomas[indices[i]]
            padre2 = cromosomas[indices[i+1]]

            if random.uniform(0,1) >= prob_cruza:
                h1, h2 = pmx(padre1, padre2)
                hijos.append(h1)
                hijos.append(h2)
                        
            
        except:
            pass
        
        i+=2


    if len(hijos) == 0:
        return []
    else:

        out = list()

        for i in hijos:
            if fPrueba == 0:
                
                fx = conteoExitos(i)
            elif fPrueba == 1:
                fx = errorMinimo(i)

            out.append([i, fx])

        return out
    

<h2>Implementación del algoritmo génetico</h2>

In [17]:
def algoritmoGenetico(tamPoblacion, n, porcCruza, porcMuta, funcionPrueba):
  padres = poblacionInicial(tamPoblacion, n,funcionPrueba)

  mejores = list()
  peores = list()
  promedio = list()

  fit = 1000000000
  contadorGeneraciones = 0
  gen = 0

  # Inicialización de la memoria
  memoria = set()

  while True:
    gen=gen+1
    print("--------------------------------")
    print("No. de Generacion=")
    print(gen)
    print(f'Contador de generaciones con el mismo mejor individio {contadorGeneraciones}')
    print(f'Mejor solución actual: {fit}')

    indices = seleccionPadres(padres)
    hijos = cruza(padres,indices, porcCruza, funcionPrueba)

    hijos = mutacion(hijos, porcMuta, funcionPrueba)
    nuevaPoblacion = padres + hijos
  
    if funcionPrueba == 0:
      sobrevivientes = sorted(nuevaPoblacion, key=itemgetter(1), reverse=True)   
    elif funcionPrueba == 1:
      sobrevivientes = sorted(nuevaPoblacion, key=itemgetter(1)) 

    padres = sobrevivientes[0: tamPoblacion]

    prom = 0
    for p in padres:
      prom += p[1]
    promedio.append(prom/len(padres))

    mejores.append(padres[0][1])
    peores.append(padres[-1][1])

    fitAnterior = fit

    fit = padres[0][1]
    print(f'Mejor solución actual: {fit}')


    print ("mejor solucion", padres[0])

    if fitAnterior == fit:
      contadorGeneraciones += 1
      if contadorGeneraciones > 20:

        tamPoblacionEstancada = int(tamPoblacion*0.2)

        PoblacionEstancada = padres[:tamPoblacionEstancada]

        poblacionAleatoria = poblacionInicial(int(tamPoblacion*0.8), n,funcionPrueba)
        
        padres = PoblacionEstancada+poblacionAleatoria
    else:
      contadorGeneraciones = 0
        
    

    if funcionPrueba == 0 and fit == ((n*2)+2):
      break
    elif funcionPrueba == 1 and fit == 0:
      break
    else:
      pass

  return mejores, peores, promedio, gen

In [18]:
algoritmoGenetico(5000,5,0.7,0.2,1)

--------------------------------
No. de Generacion=
1
Contador de generaciones con el mismo mejor individio 0
Mejor solución actual: 1000000000
Mejor solución actual: 47.0
mejor solucion [array([ 7,  5, 19, 16, 22, 17, 25, 11,  8,  2, 14, 12,  3, 23, 10,  1,  9,
       13, 20, 24, 21,  6, 18,  4, 15]), 47.0]
--------------------------------
No. de Generacion=
2
Contador de generaciones con el mismo mejor individio 0
Mejor solución actual: 47.0
Mejor solución actual: 47.0
mejor solucion [array([ 7,  5, 19, 16, 22, 17, 25, 11,  8,  2, 14, 12,  3, 23, 10,  1,  9,
       13, 20, 24, 21,  6, 18,  4, 15]), 47.0]
--------------------------------
No. de Generacion=
3
Contador de generaciones con el mismo mejor individio 1
Mejor solución actual: 47.0
Mejor solución actual: 47.0
mejor solucion [array([ 7,  5, 19, 16, 22, 17, 25, 11,  8,  2, 14, 12,  3, 23, 10,  1,  9,
       13, 20, 24, 21,  6, 18,  4, 15]), 47.0]
--------------------------------
No. de Generacion=
4
Contador de generaciones con